In [26]:
%ls

ptbxl/


In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
%cd drive

[Errno 2] No such file or directory: 'drive'
/content/drive/MyDrive/ISP


In [29]:
%cd MyDrive
%ls

[Errno 2] No such file or directory: 'MyDrive'
/content/drive/MyDrive/ISP
ptbxl/


In [30]:
%cd ISP
%ls

[Errno 2] No such file or directory: 'ISP'
/content/drive/MyDrive/ISP
ptbxl/


In [31]:
!pip install wfdb

In [36]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import wfdb
import ast

In [37]:
path = 'ptbxl/'
sampling = 100
Y = pd.read_csv(path+'ptbxl_database.csv', index_col = 'ecg_id')

In [38]:
Y

,patient_id,age,sex,height,weight,nurse,site,device,recording_date,report,...,validated_by_human,baseline_drift,static_noise,burst_noise,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr
ecg_id,,,,,,,,,,,,,,,,,,,,,
1,15709.0,56.0,1,NaN,63.0,2.0,0.0,CS-12 E,1984-11-09 09:17:34,sinusrhythmus periphere niederspannung,...,True,NaN,", I-V1,",NaN,NaN,NaN,NaN,3,records100/00000/00001_lr,records500/00000/00001_hr
2,13243.0,19.0,0,NaN,70.0,2.0,0.0,CS-12 E,1984-11-14 12:55:37,sinusbradykardie sonst normales ekg,...,True,NaN,NaN,NaN,NaN,NaN,NaN,2,records100/00000/00002_lr,records500/00000/00002_hr
3,20372.0,37.0,1,NaN,69.0,2.0,0.0,CS-12 E,1984-11-15 12:49:10,sinusrhythmus normales ekg,...,True,NaN,NaN,NaN,NaN,NaN,NaN,5,records100/00000/00003_lr,records500/00000/00003_hr
4,17014.0,24.0,0,NaN,82.0,2.0,0.0,CS-12 E,1984-11-15 13:44:57,sinusrhythmus normales ekg,...,True,", II,III,AVF",NaN,NaN,NaN,NaN,NaN,3,records100/00000/00004_lr,records500/00000/00004_hr
5,17448.0,19.0,1,NaN,70.0,2.0,0.0,CS-12 E,1984-11-17 10:43:15,sinusrhythmus normales ekg,...,True,", III,AVR,AVF",NaN,NaN,NaN,NaN,NaN,4,records100/00000/00005_lr,records500/00000/00005_hr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21833,17180.0,67.0,1,NaN,NaN,1.0,2.0,AT-60 3,2001-05-31 09:14:35,ventrikulÄre extrasystole(n) sinustachykardie ...,...,True,NaN,", alles,",NaN,NaN,1ES,NaN,7,records100/21000/21833_lr,records500/21000/21833_hr
21834,20703.0,93.0,0,NaN,NaN,1.0,2.0,AT-60 3,2001-06-05 11:33:39,sinusrhythmus lagetyp normal qrs(t) abnorm ...,...,True,NaN,NaN,NaN,NaN,NaN,NaN,4,records100/21000/21834_lr,records500/21000/21834_hr
21835,19311.0,59.0,1,NaN,NaN,1.0,2.0,AT-60 3,2001-06-08 10:30:27,sinusrhythmus lagetyp normal t abnorm in anter...,...,True,NaN,", I-AVR,",NaN,NaN,NaN,NaN,2,records100/21000/21835_lr,records500/21000/21835_hr


In [39]:
class EcgDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.diagnosis_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.diagnosis_frame)

    def __getitem__(self, idx):
        '''
        if torch.is_tensor(idx):
            idx = idx.tolist()
        '''

        ecg_file_name = os.path.join(self.root_dir, self.diagnosis_frame.iloc[1].filename_lr)
        #pdb.set_trace()
        ecg = wfdb.rdsamp(ecg_file_name)
        ecg = ecg[0]
        if ('AFIB' in self.diagnosis_frame.scp_codes[idx]):
            diagnosise = 1.0
        else:
            diagnosise = 0.0
        sample = {'ecg' : ecg, 'diagnosis': diagnosise}
        return sample

In [40]:
import pdb
%ls

ptbxl/


In [41]:
ecg_dataset = EcgDataset(path + 'ptbxl_database.csv', path)

print(ecg_dataset[2])

{'ecg': array([[ 0.004,  0.138,  0.134, ...,  0.192,  0.083,  0.088],
       [-0.02 ,  0.116,  0.136, ...,  0.156,  0.057,  0.063],
       [-0.053,  0.092,  0.145, ...,  0.107,  0.013,  0.022],
       ...,
       [ 0.121,  0.398,  0.277, ..., -1.065, -0.492, -0.156],
       [-0.326,  0.057,  0.383, ..., -0.28 , -0.175, -0.071],
       [-0.348, -0.056,  0.292, ..., -0.308, -0.231, -0.145]]), 'diagnosis': 0.0}


In [42]:
dataloader = DataLoader(ecg_dataset, batch_size = 100, shuffle=True,
                        num_workers = 0)

In [43]:
'''
def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    return data

path = 'ptbxl/'
#path = '/SystemProgramming/Task1/ptbxl/'
sampling_rate=100

# load and convert annotation data
print(path+'ptbxl_database.csv')
Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))



# Load raw signal data
X = load_raw_data(Y, sampling_rate, path)
'''

"\ndef load_raw_data(df, sampling_rate, path):\n    if sampling_rate == 100:\n        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]\n    else:\n        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]\n    data = np.array([signal for signal, meta in data])\n    return data\n\npath = 'ptbxl/'\n#path = '/SystemProgramming/Task1/ptbxl/'\nsampling_rate=100\n\n# load and convert annotation data\nprint(path+'ptbxl_database.csv')\nY = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')\nY.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))\n\n\n\n# Load raw signal data\nX = load_raw_data(Y, sampling_rate, path)\n"

In [44]:
'''
# Load scp_statements.csv for diagnostic aggregation
agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]

def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

# Apply diagnostic superclass
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

# Split data into train and test
test_fold = 10

# Train
X_train = X[np.where(Y.strat_fold != test_fold)]
X_train = torch.tensor(X_train).float().permute(0,2,1)
y = Y[(Y.strat_fold != test_fold)].diagnostic_superclass
y_train = []
for x in y:
    if ('AFIB' in x):
        y_train.append(1)
    else:
        y_train.append(0)
y_train = torch.tensor(y_train)


# Test
X_test = X[np.where(Y.strat_fold == test_fold)]
X_test = torch.tensor(X_test).float().permute(0,2,1)
y = Y[Y.strat_fold == test_fold].diagnostic_superclass
y_test = []
for x in y:
    if ('AFIB' in x):
        y_test.append(1)
    else:
        y_test.append(0)
y_test = torch.tensor(y_test)
'''

"\n# Load scp_statements.csv for diagnostic aggregation\nagg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)\nagg_df = agg_df[agg_df.diagnostic == 1]\n\ndef aggregate_diagnostic(y_dic):\n    tmp = []\n    for key in y_dic.keys():\n        if key in agg_df.index:\n            tmp.append(agg_df.loc[key].diagnostic_class)\n    return list(set(tmp))\n\n# Apply diagnostic superclass\nY['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)\n\n# Split data into train and test\ntest_fold = 10\n\n# Train\nX_train = X[np.where(Y.strat_fold != test_fold)]\nX_train = torch.tensor(X_train).float().permute(0,2,1)\ny = Y[(Y.strat_fold != test_fold)].diagnostic_superclass\ny_train = []\nfor x in y:\n    if ('AFIB' in x):\n        y_train.append(1)\n    else:\n        y_train.append(0)\ny_train = torch.tensor(y_train)\n\n\n# Test\nX_test = X[np.where(Y.strat_fold == test_fold)]\nX_test = torch.tensor(X_test).float().permute(0,2,1)\ny = Y[Y.strat_fold == test_fold].diagnostic_super

In [45]:


class ResidualBlock(nn.Module):
    def __init__(self, in_channel, out_channel, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(in_channel, out_channel, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm1d(out_channel),
            nn.ReLU()
        )

        self.conv2 = nn.Sequential(
            nn.Conv1d(out_channel, out_channel, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm1d(out_channel),
        )

        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channel = out_channel

    def forward(self, input):
        residual = input
        output = self.conv1(input)
        output = self.conv2(output)
        #pdb.set_trace()
        if (self.downsample):
            residual = self.downsample(input)

        output = output + residual
        output = self.relu(output)
        return output

class ResNet1d(nn.Module):
    def __init__(self, block, layers, num_classes=1):
        super(ResNet1d, self).__init__()

        self.inplanes = 64
        self.conv1 = nn.Sequential(
            nn.Conv1d(12, 64, kernel_size=7, stride = 2, padding = 3),
            nn.BatchNorm1d(64),
            nn.ReLU()
        )
        self.maxpool = nn.MaxPool1d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2)

        self.avgpool = nn.AvgPool1d(7, stride=1)
        self.fc = nn.Linear(512, num_classes)
        self.softmax = nn.Softmax(dim = 1)

    def _make_layer(self, block, planes, blocks, stride = 1):
        downsample = None
        if stride != 1 or self.inplanes != planes:
            #pdb.set_trace()
            downsample = nn.Sequential(
                nn.Conv1d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm1d(planes),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)

        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        #x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

    def predict(self,x):
        x = self.forward(x)
        x = self.softmax(x)
        return x
# Learning



In [46]:
resnet = ResNet1d(ResidualBlock, [3, 4, 6, 3])

model = ResNet1d(ResidualBlock, [3,4,6,3])
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(resnet.parameters(), lr=0.001)
num_epochs = 10

for epoch in range(num_epochs):
    for i_batch, sample_batched in enumerate(dataloader):
        optimizer.zero_grad()
        X_batch = sample_batched['ecg'].float()
        X_batch = X_batch.permute(0,2,1)
        y_batch = sample_batched['diagnosis']
        preds = resnet.predict(X_batch)
        loss_value = criterion(preds, y_batch)
        loss_value.backward()
        optimizer.step()
        break

    '''
    for start_index in range(0, len(X_train), batch_size):
        optimizer.zero_grad()

        batch_indexes = order[start_index:start_index+batch_size]

        X_batch = X_train[batch_indexes] #.to(device)
        y_batch = y_train[batch_indexes] #.to(device)
        #pdb.set_trace()
        preds = resnet.forward(X_batch)

        loss_value = criterion(preds, y_batch)
        loss_value.backward()

        optimizer.step()

    test_preds = mnist_net.forward(X_test)
#     test_loss_history.append(loss(test_preds, y_test))

    accuracy = (test_preds.argmax(dim=1) == y_test).float().mean()
#     test_accuracy_history.append(accuracy)
    print(accuracy)
'''

RuntimeError: The size of tensor a (250) must match the size of tensor b (125) at non-singleton dimension 2

In [ ]:
'''
resnet = ResNet1d(ResidualBlock, [3, 4, 6, 3])

num_classes = 1
num_epochs = 20
batch_size = 10
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(resnet.parameters(), lr=0.001)


for epoch in range(num_epochs):
    order = np.random.permutation(len(X_train))

    for start_index in range(0, len(X_train), batch_size):
        optimizer.zero_grad()

        batch_indexes = order[start_index:start_index+batch_size]

        X_batch = X_train[batch_indexes] #.to(device)
        y_batch = y_train[batch_indexes] #.to(device)
        #pdb.set_trace()
        preds = resnet.forward(X_batch)

        loss_value = criterion(preds, y_batch)
        loss_value.backward()

        optimizer.step()


    test_preds = mnist_net.forward(X_test)
#     test_loss_history.append(loss(test_preds, y_test))

    accuracy = (test_preds.argmax(dim=1) == y_test).float().mean()
#     test_accuracy_history.append(accuracy)
    print(accuracy)

#accuracy in metrics

with torch.no_grad():
    predictions = resnet(X_test).squeeze()
    predicted_labels = (predictions > 0.5).float()

roc_auc = roc_auc_score(y_test, predicted_labels)
f1 = f1_score(target, predicted_labels)
sensitivity = recall_score(target, predicted_labels)

print(f'ROC-AUC: {roc_auc:.4f}')
print(f'F1-score: {f1:.4f}')
print(f'Sensitivity (Recall): {sensitivity:.4f}')

'''